# Spotify Scraping

Importiamo le librerie necessarie:

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

Leggiamo il datframe di canzoni ottenuto usando l'API di Spotify:

In [ ]:
df = pd.read_csv("songs_final.csv")

Considero solo le colonne ``Artist``,``Url``,``Track`` e ``Album``

In [ ]:
df.drop(['Unnamed: 0'], axis=1,inplace=True)
df = df[['Artist' , 'Url' , 'Track', 'Album' ]]
df

Nel nostro dataframe ``df`` abbiamo una colonna chiamata ``'Url'`` che memorizza l'indirizzo web delle pagine degli artisti di Spotify. Poiché abbiamo più canzoni per ogni artista, possiamo salvarle in un elenco ottenendo i valori unici di ``'Url'``. Facciamo lo stesso con ``Artist``. 

In [ ]:
art= list(df['Artist'].unique())
url= list(df['Url'].unique())
len(art)

Creiamo le liste vuote:

In [ ]:
stream=[]
name=[]
remove=[]

Il seguente codice tilizza la libreria Selenium per automatizzare l'interazione con Open Spotify.

Il codice crea un oggetto ``options`` di tipo Options e imposta la strategia di caricamento della pagina su "normal". Quindi, viene creato un oggetto ``driver`` di tipo webdriver.Chrome utilizzando le opzioni specificate.

Successivamente, viene caricata la pagina iniziale di Open Spotify e cliccato sul pulsante "Accetta i cookie".

Nel ciclo for, per ogni URL presente nella lista ``url``, viene caricata la pagina dell'artista corrispondente e cliccato sul pulsante "see more". Quindi, vengono recuperati i titoli e i conteggi dello streaming utilizzando la selezione CSS. Se la lunghezza di elementi recuperati è inferiore a 10, viene stampato un messaggio di errore e si aggiunge il nome dell'artista nella lista ``remove`` . Altrimenti, i titoli e i conteggi dello streaming vengono salvati nella lista name e stream, rispettivamente.

In [ ]:
options = Options()
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(options=options)

driver.get("https://open.spotify.com")

WebDriverWait(driver, 120).until(EC.element_to_be_clickable((By.XPATH, "//button[@id='onetrust-accept-btn-handler']"))).click()
for i in range(0,len(url)):

        driver.get(url[i])
        WebDriverWait(driver, 120).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='Type__TypeElement-sc-goli3j-0 drxPnz']"))).click()
        elem=driver.find_elements(By.CSS_SELECTOR, ".Type__TypeElement-sc-goli3j-0.hGXzYa.nYg_xsOVmrVE_8qk1GCW")
        elem1=driver.find_elements(By.CSS_SELECTOR, ".Type__TypeElement-sc-goli3j-0.kHHFyx.t_yrXoUO3qGsJS4Y6iXX.standalone-ellipsis-one-line")
        if(len(elem) or len(elem1)) < 10:
            print("problema con",i)
            remove.append(art[i])
        else:
            for value in elem:
                stream.append(value.text)
            for value in elem1:
                name.append(value.text)
                

        

Rimuovo gli artisti per cui il codice mi ha restituito un errore:

In [ ]:
df.set_index('Artist' , inplace=True)
for i in remove:
    df.drop([i] , axis=0 ,inplace=True)


In [ ]:
df.reset_index(inplace=True)
df['Artist']

In [ ]:
df_stream=pd.DataFrame({"Artist":df['Artist'],"Url": df['Url'], "Track_Api": df['Track'] , "Album_Api":df['Album'],
                        "Stream":stream, "Track":name})
df_stream

Infine carichiamo il dataframe in un file csv:

In [ ]:
df_stream.to_csv("StreamCount_final.csv")